# 2023_07_07

transformation이 for문 기반이라 느림  
convoluation 기반으로 변경  

concate_loss 
loss를 둘이 곱하는 식으로 변경  
둘 사이의 범위가 일치하지 않아서 f1 loss가 커진는거에 따라서 qunantile loss도 커저야함  
이를 반영하기 위해서 곱하는 형식으로 변경  

loss는 1e-4가 가장 베스트임  
많이 떨어지는가 중요한게 아니라 얼마나 줄일 수 있나가 중요함  

# convolution

기존의 transformation 방식은 for문 기반으로 peak위치에 대한 정보를 저장 후  
이에 대해서 for문을 기반으로 값을 넣어주는 방식임  

python이 for문을 돌리면 성능이 안좋아지는 그러한 한계점이 존재함  
Possion tranfromation으로 Dataloader를 구성을 했을 떄 kernel이 나감  

위 방식은 일종의 합성곱(convolution)으로 생각 할 수 있음  

일종의 lagging 방식임  

lagging의 shape은 삼각형과 지수적 감소 하는 모양  
lagging의 size는 3 , 5 ,7
로 설정하고 샘플 테스트 후에 

TFT 모델에 적용 후 성능 비교  

-----------------------------------------------------------------------------------

numpy를 이용해서 conv 구현  
conv 하면서 filter size // 2 만큼 shift 되므로 이를 다시 왼쪽 shift 적용  


``` python
def triangle_conv(filter, time_series):
    size = len(filter)
    shift = size // 2
    conv_result = np.convolve(time_series, filter, mode='same')
    conv_result = np.roll(conv_result, shift)
    return conv_result
```
> test notebook : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Spatial%20_Sparsity/convolution_TEST.ipynb

위 노트북에서 현재 데이터 분포에 대해서 가정 후 실제로 conv이 어떻게 진행되는지 확인용 노트북  

> 학습용 .py : http://202.31.200.194:8888/edit/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Spatial%20_Sparsity/BaseModel_conv.py

위 파일을 이용해서 학습 진행  

> http://202.31.200.194:8888/tree/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Spatial%20_Sparsity/convolution_TEST_html

위 폴더에 html 파일 저장  

lr을 1e-4로 고정했을 때 다음과 같은 결과가 나옴  

|           | val_loss | MAE      |
|-----------|----------|----------|
| BaseModel | 0.002927 | 0.002935 |
| ConvModel | 0.002835 | 0.003081 |

conv를 적용했을 때 수치적으로 많이 개선이 되는 것 같지 않음  
conv 적용으로 인해서 데이터의 패턴이 바뀌었지만 이에 따라서 lr가 바뀌지 않음  
최적의 결과를 내기 위해서 conv 적용을했을 때 가장 성능이 좋은 상황에 대해서 다시 lr tuning을 진행  

### 결과 비교

> tensorboard : http://202.31.200.70:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Spatial%20_Sparsity/tensorboard_Long%20Beach_5.ipynb#

lr : 1e-4에서
BaseModel과 Convolution 적용한 모델의 예측값을 비교함  

In [1]:
import os
os.listdir('../Traffic_Accient/TFT/Spatial _Sparsity/conv_test/5/')

In [2]:
os.listdir('../Traffic_Accient/TFT/Spatial _Sparsity/ControlGroup/0.0001/')

In [3]:
from IPython.display import IFrame

html_file_path = '../Traffic_Accient/TFT/Spatial _Sparsity/conv_test/5/3.html'
IFrame(src=html_file_path, width='100%', height='500px')


In [4]:
from IPython.display import IFrame

html_file_path = '../Traffic_Accient/TFT/Spatial _Sparsity/ControlGroup/0.0001/3.html'
IFrame(src=html_file_path, width='100%', height='500px')

### Attention 비교


* base model attention
![](../Meeting_Img/2023_07_07/base_attetion.png)

* conv model attention
![](../Meeting_Img/2023_07_07/conv_attetion.png)

-50 부분에서 attention 값이 크게 뜀  
뿐만 아니라 attetnion 값이 다른 곳에 비교해서 크게 뛰는 구간이 존재함  

이때에 어떤 패턴을 가지는지 확인이 필요함  

### decoder 비교


* base model decoder importance
![](../Meeting_Img/2023_07_07/base_decoder_importance.png)

* conv model decoder importance
![](../Meeting_Img/2023_07_07/conv_decoder_importance.png)

decoder는 크게 순서가 뒤 바뀜  

### Encoder 비교


* base model encoder importance
![](../Meeting_Img/2023_07_07/base_encoder_importance.png)

* conv model encoder importance
![](../Meeting_Img/2023_07_07/conv_encoder_importance.png)

encoder는 크게 순서가 뒤 바뀜  

### val attention 비교


* base model val importance
![](../Meeting_Img/2023_07_07/base_val_importance.png)

* conv model val importance
![](../Meeting_Img/2023_07_07/conv_val_importance.png)

val는 크게 순서가 뒤 바뀜  

### val attention 비교


* base model val decoder importance
![](../Meeting_Img/2023_07_07/base_val_decoder_importance.png)

* conv model val decoder importance
![](../Meeting_Img/2023_07_07/conv_val_decoder_importance.png)

val는 크게 순서가 뒤 바뀜  

### Month prediction

> controlGroup_best : http://202.31.200.194:8888/view/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Spatial%20_Sparsity/ControlGroup_result/plolty_quantile/0.0001_0.0001.html

> convolution_best : http://202.31.200.194:8888/view/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Spatial%20_Sparsity/ConvolutionGroup_result/plolty_quantile/0.0001_0.0001.html

> concateloss_best : http://202.31.200.194:8888/view/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Spatial%20_Sparsity/lossGroup_result/plolty_quantile/0.0001_0.0001.html


convolution 적용시에 높이에 대해서 높이의 편차가 조금더 생김  

In [5]:
from IPython.display import IFrame


In [6]:
html_file_path = '../Traffic_Accient/TFT/Spatial _Sparsity/ControlGroup_result/plolty_quantile/0.0001_0.0001.html'
IFrame(src=html_file_path, width='100%', height='500px')

In [7]:
html_file_path = '../Traffic_Accient/TFT/Spatial _Sparsity/ConvolutionGroup_result/plolty_quantile/0.0001_0.0001.html'
IFrame(src=html_file_path, width='100%', height='500px')

In [14]:
html_file_path = '../Traffic_Accient/TFT/Spatial _Sparsity/lossGroup_result/plolty_quantile/0.0001_0.0001.html'
IFrame(src=html_file_path, width='100%', height='500px')

In [22]:
import pandas as pd
pd.read_csv('../Traffic_Accient/TFT/Spatial _Sparsity/ConvolutionGroup_result/csv/0.0001_0.0001.csv')

,Unnamed: 0,Quantile Loss,MSE,Accuracy,Recall,Precision,F1 Score,F2 Score,GeoHash
0,0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9mgzy
1,1,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9mupb
2,2,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9q5bj
3,3,0.002016,0.004032,0.995968,0.914286,1.0,0.955224,0.930233,9q5bm
4,4,0.002016,0.004032,0.995968,0.785714,1.0,0.880000,0.820896,9q5bn
5,5,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9q5bp
6,6,0.006048,0.012097,0.987903,0.763158,1.0,0.865672,0.801105,9q5bq
7,7,0.002688,0.005376,0.994624,0.851852,1.0,0.920000,0.877863,9q5br
8,8,0.006048,0.012097,0.987903,0.808511,1.0,0.894118,0.840708,9q5bw
9,9,0.000672,0.001344,0.998656,0.928571,1.0,0.962963,0.942029,9q5bx


In [23]:
import pandas as pd
pd.read_csv('../Traffic_Accient/TFT/Spatial _Sparsity/ControlGroup_result/csv/0.001_0.001.csv')

,Unnamed: 0,Quantile Loss,MSE,Accuracy,Recall,Precision,F1 Score,F2 Score,GeoHash
0,0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9mgzy
1,1,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9mupb
2,2,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9q5bj
3,3,0.000672,0.001344,0.998656,0.971429,1.0,0.985507,0.977011,9q5bm
4,4,0.002016,0.004032,0.995968,0.785714,1.0,0.880000,0.820896,9q5bn
5,5,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9q5bp
6,6,0.005376,0.010753,0.989247,0.789474,1.0,0.882353,0.824176,9q5bq
7,7,0.002688,0.005376,0.994624,0.851852,1.0,0.920000,0.877863,9q5br
8,8,0.006048,0.012097,0.987903,0.808511,1.0,0.894118,0.840708,9q5bw
9,9,0.000000,0.000000,1.000000,1.000000,1.0,1.000000,1.000000,9q5bx


In [24]:
import pandas as pd
pd.read_csv('../Traffic_Accient/TFT/Spatial _Sparsity/lossGroup_result/csv/0.001_0.001.csv')

,Unnamed: 0,Quantile Loss,MSE,Accuracy,Recall,Precision,F1 Score,F2 Score,GeoHash
0,0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9mgzy
1,1,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9mupb
2,2,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9q5bj
3,3,0.000672,0.001344,0.998656,0.971429,1.0,0.985507,0.977011,9q5bm
4,4,0.002016,0.004032,0.995968,0.785714,1.0,0.880000,0.820896,9q5bn
5,5,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,9q5bp
6,6,0.005376,0.010753,0.989247,0.789474,1.0,0.882353,0.824176,9q5bq
7,7,0.002688,0.005376,0.994624,0.851852,1.0,0.920000,0.877863,9q5br
8,8,0.006048,0.012097,0.987903,0.808511,1.0,0.894118,0.840708,9q5bw
9,9,0.000000,0.000000,1.000000,1.000000,1.0,1.000000,1.000000,9q5bx


### Custom loss

quantile loss와 f1 loss를 곱한 loss를 사용  
이상하게 custom loss를 사용하면 out_dim이 1로 되는 현상이 발생  

다음과 같이 Custom loss를 정의함 


```python
class CustomLoss(MultiHorizonMetric):
    """
    Quantile loss, i.e. a quantile of ``q=0.5`` will give half of the mean absolute error as it is calculated as

    Defined as ``max(q * (y-y_pred), (1-q) * (y_pred-y))``
    """

    def __init__(
        self,
        # f1 loss를 계산시에 discrete한 값에서 사용 할 수 있으므로 이를 만들기 위한 threshold 
        threshold: float = 0.5,
        quantiles: List[float] = [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98],
        **kwargs,
    ):
        """
        Quantile loss

        Args:
            quantiles: quantiles for metric
        """
        self.threshold = threshold
        super().__init__(quantiles=quantiles, **kwargs)

    def loss(self, y_pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        # calculate quantile loss
        losses = []
        for i, q in enumerate(self.quantiles):
            errors = target - y_pred[..., i]
            # 기존의 quantile loss * f1 loss
            losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(-1) * (self.calculate_f1_loss(target , - y_pred[..., i])))
        losses = 2 * torch.cat(losses, dim=2)

        return losses
    
    # fl loss 계산식
    def calculate_f1_loss(self, y_true: torch.Tensor, y_pred: torch.Tensor) -> torch.Tensor:
        y_true_binary = (y_true > self.threshold).float()
        #y_pred = self.to_prediction(y_pred)
        y_pred_binary = (y_pred > self.threshold).float()

        tp = torch.sum(y_true_binary * y_pred_binary, dim=0)
        fp = torch.sum((1 - y_true_binary) * y_pred_binary, dim=0)
        fn = torch.sum(y_true_binary * (1 - y_pred_binary), dim=0)

        epsilon = 1e-7  # to prevent division by zero
        precision = tp / (tp + fp + epsilon)
        recall = tp / (tp + fn + epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + epsilon)
        return 1 - f1.mean()  # return 1 minus mean f1 score to make it a loss (lower is better)


    def to_prediction(self, y_pred: torch.Tensor) -> torch.Tensor:
        """
        Convert network prediction into a point prediction.

        Args:
            y_pred: prediction output of network

        Returns:
            torch.Tensor: point prediction
        """
        if y_pred.ndim == 3:
            idx = self.quantiles.index(0.5)
            y_pred = y_pred[..., idx]
        return y_pred

    def to_quantiles(self, y_pred: torch.Tensor) -> torch.Tensor:
        """
        Convert network prediction into a quantile prediction.

        Args:
            y_pred: prediction output of network

        Returns:
            torch.Tensor: prediction quantiles
        """
        return y_pred
```

이후에 다음과 같이 TFT 변경

```python
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=1e-5,
    hidden_size=32,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=32,
    loss=CustomLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    optimizer="Ranger",
    reduce_on_plateau_patience=4,
    # output size를 설정
    output_size= 7
)
```

custom loss 확인용 파일  
> http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Spatial%20_Sparsity/concate_loss.ipynb#